# Day 4
## Camp Cleanup
Space needs to be cleared before the last supplies can be unloaded from the ships, and so several Elves have been assigned the job of cleaning up sections of the camp. Every section has a unique ID number, and each Elf is assigned a range of section IDs.

However, as some of the Elves compare their section assignments with each other, they've noticed that many of the assignments overlap. To try to quickly find overlaps and reduce duplicated effort, the Elves pair up and make a big list of the section assignments for each pair (your puzzle input).

For example, consider the following list of section assignment pairs:

```
2-4,6-8
2-3,4-5
5-7,7-9
2-8,3-7
6-6,4-6
2-6,4-8
```
For the first few pairs, this list means:

- Within the first pair of Elves, the first Elf was assigned sections 2-4 (sections 2, 3, and 4), while the second Elf was assigned sections 6-8 (sections 6, 7, 8).
- The Elves in the second pair were each assigned two sections.
- The Elves in the third pair were each assigned three sections: one got sections 5, 6, and 7, while the other also got 7, plus 8 and 9.
This example list uses single-digit section IDs to make it easier to draw; your actual list might contain larger numbers. Visually, these pairs of section assignments look like this:
```
.234.....  2-4
.....678.  6-8

.23......  2-3
...45....  4-5

....567..  5-7
......789  7-9

.2345678.  2-8
..34567..  3-7

.....6...  6-6
...456...  4-6

.23456...  2-6
...45678.  4-8
```
Some of the pairs have noticed that one of their assignments fully contains the other. For example, 2-8 fully contains 3-7, and 6-6 is fully contained by 4-6. In pairs where one assignment fully contains the other, one Elf in the pair would be exclusively cleaning sections their partner will already be cleaning, so these seem like the most in need of reconsideration. In this example, there are 2 such pairs.

**In how many assignment pairs does one range fully contain the other?**

## Part 1

### Read data and build working dataframe

In [1]:
import pandas as pd

In [4]:
sections_df = pd.read_csv("cleanup.txt", header=None)
sections_df.columns = ["elf1", "elf2"]

In [5]:
sections_df

,elf1,elf2
0,75-76,18-75
1,2-54,1-50
2,82-83,78-82
3,13-37,37-75
4,79-80,2-80
...,...,...
995,19-96,95-99
996,39-74,40-87
997,64-71,71-72
998,49-80,36-80


### Convert sections to ranges

In [38]:
def create_range(section):
    a = section.split("-")
    return [i for i in range(int(a[0]), int(a[1]) + 1)]

### Identify if section contains the other

In [51]:
sections_df["range1contains2"] = sections_df.apply(lambda x: all(item in create_range(x["elf1"]) for item in create_range(x["elf2"])), axis=1)
sections_df["range2contains1"] = sections_df.apply(lambda x: all(item in create_range(x["elf2"]) for item in create_range(x["elf1"])), axis=1)

In [52]:
sections_df

,elf1,elf2,range1contains2,range2contains1
0,75-76,18-75,False,False
1,2-54,1-50,False,False
2,82-83,78-82,False,False
3,13-37,37-75,False,False
4,79-80,2-80,False,True
...,...,...,...,...
995,19-96,95-99,False,False
996,39-74,40-87,False,False
997,64-71,71-72,False,False
998,49-80,36-80,False,True


### How many?

In [53]:
# sections_df[sections_df["range1contains2"] & ~sections_df["range2contains1"]]  # 246
# sections_df[~sections_df["range1contains2"] & sections_df["range2contains1"]]  # 238
# sections_df[sections_df["range1contains2"] & sections_df["range2contains1"]]  # 12
# print(246+238+12)  # 496
sections_df[sections_df["range1contains2"] | sections_df["range2contains1"]]  # 496

,elf1,elf2,range1contains2,range2contains1
4,79-80,2-80,False,True
5,29-90,30-89,True,False
7,20-93,20-94,False,True
10,42-52,15-52,False,True
13,18-81,17-81,False,True
...,...,...,...,...
990,15-28,15-47,False,True
991,14-64,15-64,True,False
992,88-88,25-89,False,True
993,80-80,15-80,False,True


#### 496
That's the right answer! You are one gold star closer to collecting enough star fruit.

---

## Part 2

It seems like there is still quite a bit of duplicate work planned. Instead, the Elves would like to know the number of pairs that overlap at all.

In the above example, the first two pairs (2-4,6-8 and 2-3,4-5) don't overlap, while the remaining four pairs (5-7,7-9, 2-8,3-7, 6-6,4-6, and 2-6,4-8) do overlap:

- 5-7,7-9 overlaps in a single section, 7.
- 2-8,3-7 overlaps all of the sections 3 through 7.
- 6-6,4-6 overlaps in a single section, 6.
- 2-6,4-8 overlaps in sections 4, 5, and 6.

So, in this example, the number of overlapping assignment pairs is 4.

**In how many assignment pairs do the ranges overlap?**

In [84]:
sections_df["any1in2"] = sections_df.apply(lambda x: any(item in create_range(x["elf1"]) for item in create_range(x["elf2"])), axis=1)

In [85]:
sections_df

,elf1,elf2,range1contains2,range2contains1,any1in2
0,75-76,18-75,False,False,True
1,2-54,1-50,False,False,True
2,82-83,78-82,False,False,True
3,13-37,37-75,False,False,True
4,79-80,2-80,False,True,True
...,...,...,...,...,...
995,19-96,95-99,False,False,True
996,39-74,40-87,False,False,True
997,64-71,71-72,False,False,True
998,49-80,36-80,False,True,True


In [88]:
# No overlap at all
sections_df[~sections_df["any1in2"]]  # 153

,elf1,elf2,range1contains2,range2contains1,any1in2
16,24-89,7-11,False,False,False
37,10-81,9-9,False,False,False
38,3-40,58-73,False,False,False
41,7-84,6-6,False,False,False
47,44-80,13-26,False,False,False
...,...,...,...,...,...
965,43-60,33-37,False,False,False
972,11-72,2-5,False,False,False
985,24-24,25-71,False,False,False
987,1-10,29-85,False,False,False


In [92]:
# Any overlap
sections_df[sections_df["any1in2"]]  # 847

,elf1,elf2,range1contains2,range2contains1,any1in2
0,75-76,18-75,False,False,True
1,2-54,1-50,False,False,True
2,82-83,78-82,False,False,True
3,13-37,37-75,False,False,True
4,79-80,2-80,False,True,True
...,...,...,...,...,...
995,19-96,95-99,False,False,True
996,39-74,40-87,False,False,True
997,64-71,71-72,False,False,True
998,49-80,36-80,False,True,True


#### 847

That's the right answer! You are one gold star closer to collecting enough star fruit.

You have completed Day 4!